In [1]:
import os

In [2]:
%pwd

'c:\\Users\\N Ramaraju\\Desktop\\Projects\\LLM-Text-Detection\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\N Ramaraju\\Desktop\\Projects\\LLM-Text-Detection'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [6]:
from LLM_text_detection.constants import *
from LLM_text_detection.utils.common import *


In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
        )

        return data_transformation_config

In [10]:
import os
from LLM_text_detection import logger
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


[2024-06-22 11:27:16,079: INFO: utils: NumExpr defaulting to 8 threads.]


In [11]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    
    ## Note: You can add different data transformation techniques such as Scaler, PCA and all
    #You can perform all kinds of EDA in ML cycle here before passing this data to the model

    # I am only adding train_test_spliting cz this data is already cleaned up

    def Data_transformation(self):
        data = pd.read_csv(self.config.data_path)
        # Initialize the TF-IDF Vectorizer
        tfidf_vectorizer = TfidfVectorizer(max_features=5000)
        logger.info("Vectorizer is loaded")       
        # Tokenize and create TF-IDF vectors for the 'text' column of the dataset
        tfidf_vectors = tfidf_vectorizer.fit_transform(data['text'])
        logger.info("Vectorizer Transformed Data")        
        # Convert TF-IDF vectors to a DataFrame for easy analysis
        tfidf_df = pd.DataFrame(tfidf_vectors.toarray(), columns=tfidf_vectorizer.get_feature_names_out())
        logger.info("Vectorization is completed")
        # Split the data into training and testing sets
        X = tfidf_df.values  # Features (TF-IDF vectors)
        y = data['label'].values  # Target variable
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        logger.info("Spliting of Data is Completed")        
        # Encode target labels (0 and 1) using LabelEncoder
        label_encoder = LabelEncoder()
        y_train_encoded = label_encoder.fit_transform(y_train)
        y_test_encoded = label_encoder.transform(y_test)
        logger.info("Label Encoder is Done")        
        # Save the transformed data to CSV files
        pd.DataFrame(X_train).to_csv(os.path.join(self.config.root_dir,'x_train.csv'), index=False)
        pd.DataFrame(X_test).to_csv(os.path.join(self.config.root_dir,'x_test.csv'), index=False)
        pd.DataFrame(y_train_encoded).to_csv(os.path.join(self.config.root_dir,'y_train_encoded.csv'), index=False)
        pd.DataFrame(y_test_encoded).to_csv(os.path.join(self.config.root_dir,'y_test_encoded.csv'), index=False)

    def train_test_spliting(self):
        data = pd.read_csv(self.config.data_path)

        # Split the data into training and test sets. (0.75, 0.25) split.
        train, test = train_test_split(data)

        train.to_csv(os.path.join(self.config.root_dir, "train.csv"),index = False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"),index = False)

        logger.info("Splited data into training and test sets")
        logger.info(train.shape)
        logger.info(test.shape)

        print("Train Shape>>>",train.shape)
        print("Test Shape>>>",test.shape)

In [12]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.Data_transformation()
except Exception as e:
    raise e

[2024-06-22 11:27:20,526: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-06-22 11:27:20,528: INFO: common: yaml file: params.yaml loaded successfully]
[2024-06-22 11:27:20,529: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-06-22 11:27:20,531: INFO: common: created directory at: artifacts]
[2024-06-22 11:27:20,532: INFO: common: created directory at: artifacts/data_transformation]
[2024-06-22 11:27:21,670: INFO: 3711323853: Vectorizer is loaded]
[2024-06-22 11:27:30,577: INFO: 3711323853: Vectorizer Transformed Data]
[2024-06-22 11:27:34,962: INFO: 3711323853: Vectorization is completed]
[2024-06-22 11:27:45,588: INFO: 3711323853: Spliting of Data is Completed]
[2024-06-22 11:27:45,790: INFO: 3711323853: Label Encoder is Done]
